In [1]:
import pulp
import pandas as pd

In [2]:
def gerar_grade_horaria_por_turma(tabela, instancia, turma_escolhida):
    horarios = instancia["horarios"]
    horarios_legenda = instancia["horarios_legenda"]
    dias = ["Segunda", "Terça", "Quarta", "Quinta", "Sexta"]

    codigos_horario = sorted(
        {h.split("_")[1] for h in horarios},
        key=lambda x: (x[0], int(x[1]))
    )
    eixo_y = [horarios_legenda[codigo] for codigo in codigos_horario]
    grade = pd.DataFrame("", index=eixo_y, columns=dias)
    aulas_turma = tabela[tabela["turma"] == turma_escolhida]

    for _, aula in aulas_turma.iterrows():
        dia, codigo = aula["horario"].split("_")
        horario_legivel = horarios_legenda[codigo]

        conteudo = (
            f"{aula['disciplina']} - {aula['professor']} - {aula['sala']}"
        )

        grade.loc[horario_legivel, dia] = conteudo

    return grade

def exportar_grades_para_excel(tabela, instancia, caminho_arquivo_excel="grade_horaria_escola.xlsx", pasta=None):
    turmas = instancia["turmas"]

    with pd.ExcelWriter(f"{pasta}/{caminho_arquivo_excel}", engine="openpyxl") as writer:
        for turma in turmas:
            grade = gerar_grade_horaria_por_turma(tabela, instancia, turma)
            nome_aba = f"Turma_{turma}"[:31]
            grade.to_excel(writer, sheet_name=nome_aba)

    print(f"Arquivo Excel gerado: {caminho_arquivo_excel}")


In [3]:
def montar_instancia():
    turmas = ["1A", "2A", "3A"]
    ano_turma = {"1A": 1, "2A": 2, "3A": 3}

    disciplinas_obrigatorias = ["Matematica", "Fisica", "Quimica"]
    disciplinas_complementares = ["Cultura digital", "Estudo orientado"]
    disciplinas_programas_internos = ["Inovativa"]
    disciplinas_programas_externos = ["Senac", "CESAR"]
    professores = ["prof1", "prof2", "prof3", "prof4", "prof5", "prof6", "prof7"]

    horarios = ["Segunda_M1", "Segunda_M2", "Segunda_M3", "Segunda_M4", "Segunda_T1", "Segunda_T2", "Segunda_T3", "Segunda_T4", 
                "Terça_M1", "Terça_M2", "Terça_M3", "Terça_M4", "Terça_T1", "Terça_T2", "Terça_T3", "Terça_T4",
                "Quarta_M1", "Quarta_M2", "Quarta_M3", "Quarta_M4", "Quarta_T1", "Quarta_T2", "Quarta_T3", "Quarta_T4",
                "Quinta_M1", "Quinta_M2", "Quinta_M3", "Quinta_M4", "Quinta_T1", "Quinta_T2", "Quinta_T3", "Quinta_T4",
                "Sexta_M1", "Sexta_M2", "Sexta_M3", "Sexta_M4", "Sexta_T1", "Sexta_T2", "Sexta_T3", "Sexta_T4"
                ]
    horarios_legenda = {"M1": "8 às 9", "M2": "9 às 10", "M3": "10 às 11", "M4": "11 às 12", 
                        "T1": "13 às 14", "T2": "14 às 15", "T3": "15 às 16", "T4": "16 às 17"}

    salas = ["Sala_1", "Sala_2", "Sala_3"]

    professor_pode_lecionar = {
        ("prof1", "Matematica"): 1,
        ("prof2", "Fisica"): 1,
        ("prof3", "Quimica"): 1,
        ("prof4", "Inovativa"): 1,
        ("prof5", "Cultura digital"): 1,
        ("prof5", "Estudo orientado"): 1,
        ("prof6", "Senac"): 1,
        ("prof7", "CESAR"): 1,
    }

    disponibilidade = {(p, h): 1 for p in professores for h in horarios}

    quantidade_aulas = {}
    for turma in turmas:
        quantidade_aulas[(turma, "Matematica")] = 2
        quantidade_aulas[(turma, "Fisica")] = 2
        quantidade_aulas[(turma, "Quimica")] = 2

        if (ano_turma[turma] == 1):
            quantidade_aulas[(turma, "CESAR")] = 1
            quantidade_aulas[(turma, "Senac")] = 1            

    atribuicoes_fixas = {}

    return {
        "turmas": turmas,
        "ano_da_turma": ano_turma,
        "disciplinas_obrigatorias": disciplinas_obrigatorias,
        "professores": professores,
        "horarios": horarios,
        "horarios_legenda": horarios_legenda,
        "salas": salas,
        "professor_pode_lecionar": professor_pode_lecionar,
        "professor_disponivel_no_horario": disponibilidade,
        "quantidade_aulas": quantidade_aulas,
        "atribuicoes_fixas": atribuicoes_fixas,
        "disciplinas_complementares": disciplinas_complementares,
        "disciplinas_programas_internos": disciplinas_programas_internos,
        "disciplinas_programas_externos": disciplinas_programas_externos,
    }


In [ ]:
def construir_e_resolver_modelo(instancia, limite_tempo=120):
    turmas = instancia["turmas"]
    ano_da_turma = instancia["ano_da_turma"]
    disciplinas_obrigatorias = instancia["disciplinas_obrigatorias"]
    professores = instancia["professores"]
    horarios = instancia["horarios"]
    salas = instancia["salas"]

    professor_pode_lecionar = instancia["professor_pode_lecionar"]
    professor_disponivel_no_horario = instancia["professor_disponivel_no_horario"]

    quantidade_aulas = instancia["quantidade_aulas"]

    atribuicoes_fixas = set(instancia.get("atribuicoes_fixas", []))
    disciplinas_complementares = set(instancia.get("disciplinas_complementares", []))
    disciplinas_programas_internos = set(instancia.get("disciplinas_programas_internos", []))
    disciplinas_programas_externos = set(instancia.get("disciplinas_programas_externos", []))

    #  VARIÁVEIS DE DECISÃO
    #  x[turma, disciplina, professor, horario, sala] = 1 se a aula ocorre

    # Objetivo zero (somente factibilidade)
    problema = pulp.LpProblem("Alocacao_Aulas_Escolar")
    problema += 0, "Objetivo_Nulo"

    variavel_alocacao = {}

    for turma in turmas:
        for disciplina in (
            set(disciplinas_obrigatorias)
            | set(disciplinas_programas_internos)
            | set(disciplinas_programas_externos)
            | set(disciplinas_complementares)
        ):
            for professor in professores:
                for horario in horarios:
                    for sala in salas:

                        if (turma, disciplina, professor, horario, sala) in atribuicoes_fixas:
                            continue

                        if professor_pode_lecionar.get((professor, disciplina), 0) == 0:
                            continue

                        if professor_disponivel_no_horario.get((professor, horario), 0) == 0:
                            continue

                        variavel_alocacao[(turma, disciplina, professor, horario, sala)] = (
                            pulp.LpVariable(
                                f"alocar_{turma}_{disciplina}_{professor}_{horario}_{sala}",
                                cat="Binary"
                            )
                    )

    #  RESTRIÇÃO 1 - COBERTURA DAS AULAS
    disciplinas_com_quantidade = set(
        d for (t, d) in quantidade_aulas.keys()
    )
    for turma in turmas:
        for disciplina in disciplinas_com_quantidade:

            if ((turma, disciplina) in quantidade_aulas.keys()):
                problema += (
                    pulp.lpSum(
                        variavel_alocacao.get((turma, disciplina, professor, horario, sala), 0)
                        for professor in professores
                        for horario in horarios
                        for sala in salas
                    ) == quantidade_aulas[(turma, disciplina)],
                    f"cobertura_{turma}_{disciplina}"
                )

    #  RESTRIÇÃO 2 - UMA AULA POR TURMA POR HORÁRIO
    for turma in turmas:
        for horario in horarios:

            problema += (
                pulp.lpSum(
                    variavel_alocacao.get((turma, disciplina, professor, horario, sala), 0)
                    for disciplina in disciplinas_obrigatorias
                    for professor in professores
                    for sala in salas
                ) <= 1,
                f"uma_aula_por_turma_{turma}_{horario}"
            )

    #  RESTRIÇÃO 3 - UM PROFESSOR POR HORÁRIO
    for professor in professores:
        for horario in horarios:

            problema += (
                pulp.lpSum(
                    variavel_alocacao.get((turma, disciplina, professor, horario, sala), 0)
                    for turma in turmas
                    for disciplina in disciplinas_obrigatorias
                    for sala in salas
                ) <= 1,
                f"uma_aula_por_professor_{professor}_{horario}"
            )

    #  RESTRIÇÃO 4 - LIMITE DE SALAS POR HORÁRIO
    for sala in salas:
        for horario in horarios:

            problema += (
                pulp.lpSum(
                    variavel_alocacao.get((turma, disciplina, professor, horario, sala), 0)
                    for turma in turmas
                    for disciplina in disciplinas_obrigatorias
                    for professor in professores
                ) <= 1,
                f"uma_aula_por_sala_{sala}_{horario}"
            )
    
    #  RESTRIÇÕES 5 - DISCIPLINAS NÃO OBRIGATORIAS
    for turma in turmas:
        ano = ano_da_turma[turma]
        variaveis_programas_internos = pulp.lpSum(
            variavel_alocacao.get((turma, disc, professor, horario, sala), 0)
            for disc in disciplinas_programas_internos
            for professor in professores
            for horario in horarios
            for sala in salas
        )
        variaveis_programas_externos = pulp.lpSum(
            variavel_alocacao.get((turma, disc, professor, horario, sala), 0)
            for disc in disciplinas_programas_externos
            for professor in professores
            for horario in horarios
            for sala in salas
        )

        if ano == 1:
            problema += (
                variaveis_programas_internos >= 1,
                f"ano1_precisa_programa_internos_{turma}"
            )
            problema += (
                variaveis_programas_externos >= 1,
                f"ano1_precisa_programa_externos_{turma}"
            )

        if ano == 2:
            problema += (
                variaveis_programas_internos + variaveis_programas_externos <= 1,
                f"ano2_limite_programa_{turma}"
            )

        if ano == 3:
            problema += (
                variaveis_programas_internos + variaveis_programas_externos == 0,
                f"ano3_sem_programa_{turma}"
            )

    #  SOLVER
    solver = pulp.PULP_CBC_CMD(msg=True, timeLimit=limite_tempo, threads=2)
    status = problema.solve(solver)

    print("\nSTATUS DO SOLVER:", pulp.LpStatus[problema.status])

    atribuicoes = []

    # Atribuições fixas
    for (turma, disc_ext, professor, horario) in atribuicoes_fixas:
        atribuicoes.append({
            "turma": turma,
            "disciplina": disc_ext,
            "professor": professor,
            "horario": horario,
            "sala": sala,
            "fixa": True
        })

    # Atribuições decididas pelo modelo
    for chave, var in variavel_alocacao.items():
        if pulp.value(var) is not None and pulp.value(var) > 0.5:
            turma, disc_ext, professor, horario, sala = chave
            atribuicoes.append({
                "turma": turma,
                "disciplina": disc_ext,
                "professor": professor,
                "horario": horario,
                "sala": sala,
                "fixa": False
            })

    df = pd.DataFrame(atribuicoes)
    if not df.empty:
        df = df.sort_values(["horario", "turma", "disciplina"]).reset_index(drop=True)

    return problema, df

In [5]:
instancia = montar_instancia()
instancia

{'turmas': ['1A', '2A', '3A'],
 'ano_da_turma': {'1A': 1, '2A': 2, '3A': 3},
 'disciplinas_obrigatorias': ['Matematica', 'Fisica', 'Quimica'],
 'professores': ['prof1',
  'prof2',
  'prof3',
  'prof4',
  'prof5',
  'prof6',
  'prof7'],
 'horarios': ['Segunda_M1',
  'Segunda_M2',
  'Segunda_M3',
  'Segunda_M4',
  'Segunda_T1',
  'Segunda_T2',
  'Segunda_T3',
  'Segunda_T4',
  'Terça_M1',
  'Terça_M2',
  'Terça_M3',
  'Terça_M4',
  'Terça_T1',
  'Terça_T2',
  'Terça_T3',
  'Terça_T4',
  'Quarta_M1',
  'Quarta_M2',
  'Quarta_M3',
  'Quarta_M4',
  'Quarta_T1',
  'Quarta_T2',
  'Quarta_T3',
  'Quarta_T4',
  'Quinta_M1',
  'Quinta_M2',
  'Quinta_M3',
  'Quinta_M4',
  'Quinta_T1',
  'Quinta_T2',
  'Quinta_T3',
  'Quinta_T4',
  'Sexta_M1',
  'Sexta_M2',
  'Sexta_M3',
  'Sexta_M4',
  'Sexta_T1',
  'Sexta_T2',
  'Sexta_T3',
  'Sexta_T4'],
 'horarios_legenda': {'M1': '8 às 9',
  'M2': '9 às 10',
  'M3': '10 às 11',
  'M4': '11 às 12',
  'T1': '13 às 14',
  'T2': '14 às 15',
  'T3': '15 às 16',
  

In [6]:
solucao, tabela = construir_e_resolver_modelo(instancia)

print("Objetivos:", pulp.value(solucao.objective))
print("Restrições:", len(solucao.constraints))
print("Variáveis:", len(solucao.variables()))

if tabela.empty:
    print("Nenhuma atribuição encontrada — instância possivelmente inviável ou precisa de tempo maior.")
else:
    print("\nGrade encontrada:")
    print(tabela.to_string())



STATUS DO SOLVER: Optimal
Objetivos: None
Restrições: 535
Variáveis: 2161

Grade encontrada:
   turma  disciplina professor     horario    sala   fixa
0     1A      Fisica     prof2   Quarta_M1  Sala_2  False
1     1A   Inovativa     prof4   Quarta_M1  Sala_1  False
2     3A     Quimica     prof3   Quarta_M1  Sala_1  False
3     1A  Matematica     prof1   Quarta_M2  Sala_1  False
4     3A      Fisica     prof2   Quarta_M2  Sala_2  False
5     3A  Matematica     prof1   Quarta_M3  Sala_1  False
6     3A     Quimica     prof3   Quarta_M4  Sala_1  False
7     1A     Quimica     prof3   Quarta_T4  Sala_3  False
8     3A      Fisica     prof2   Quinta_M3  Sala_2  False
9     1A       CESAR     prof7   Quinta_M4  Sala_1  False
10    2A     Quimica     prof3   Quinta_T3  Sala_2  False
11    1A  Matematica     prof1   Quinta_T4  Sala_2  False
12    1A      Fisica     prof2  Segunda_T1  Sala_2  False
13    2A  Matematica     prof1  Segunda_T4  Sala_1  False
14    2A  Matematica     prof1    Se

In [7]:
gerar_grade_horaria_por_turma(tabela, instancia, instancia["turmas"][0])

,Segunda,Terça,Quarta,Quinta,Sexta
8 às 9,,,Inovativa - prof4 - Sala_1,,
9 às 10,,Senac - prof6 - Sala_3,Matematica - prof1 - Sala_1,,
10 às 11,,,,,
11 às 12,,,,CESAR - prof7 - Sala_1,
13 às 14,Fisica - prof2 - Sala_2,,,,
14 às 15,,Quimica - prof3 - Sala_3,,,
15 às 16,,,,,
16 às 17,,,Quimica - prof3 - Sala_3,Matematica - prof1 - Sala_2,


In [8]:
exportar_grades_para_excel(tabela, instancia, caminho_arquivo_excel="grade_horaria_padre_machado.xlsx", pasta="rascunhos")

Arquivo Excel gerado: grade_horaria_padre_machado.xlsx
